Imagínate que trabajas para un banco y éste tiene un `problema` importante: los clientes se marchan.

A tu equipo le han encargado entender el porqué se marchan y ver si se puede hacer algo para, `proactivamente`, ir a aquellos clientes que tienen una posibilidad alta de marcharse y ofrecerles algún producto o servicio para evitarlo.

La tarea que te han asignado es la de estudiar y entender el dataset que te han proporcionado y emplear diferentes técnicas para poder estar en las mejores condiciones para después entrenar un modelo predictivo

**Preprocesamiento de los datos y Estudio inicial:**

**Varificamos muestras**

In [1]:
import pandas as pd

# Carga del dataset
dataset_path = "data/r1_data_2024.csv"
df = pd.read_csv(dataset_path)

print("Resumen del DataFrame:\n")
df.info()

# Determinar número de muestras y variables
num_muestras = df.shape[0]
num_variables = df.shape[1]

print(f"\nEl dataset contiene {num_muestras} muestras y {num_variables} variables.")

# Determinando el número de variables categóricas y numéricas
variables_categoricas = df.select_dtypes(include=['object', 'category']).shape[1]
variables_numericas = df.select_dtypes(include=['number']).shape[1]

print(f"De las {num_variables} variables, {variables_categoricas} son categóricas "
      f"y {variables_numericas} son numéricas.")


Resumen del DataFrame:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Id                         10127 non-null  int64  
 1   ClosedAccount              10127 non-null  object 
 2   Age                        10006 non-null  float64
 3   Gender                     10016 non-null  object 
 4   Education                  10127 non-null  object 
 5   CivilState                 10012 non-null  object 
 6   Salary                     10057 non-null  object 
 7   Product                    10127 non-null  int64  
 8   NumMonthsBeingClient       10127 non-null  int64  
 9   NumProductsClient          10127 non-null  int64  
 10  NumMonthsWithoutOperating  10127 non-null  int64  
 11  CreditLimit                10025 non-null  float64
 12  CreditNotPayed             10064 non-null  float64
 13  SumTransactionAmount  

* Notamos algunos valores nulos en columnas como 'Age', 'Gender', 'CivilState', 'Salary', 'CreditLimit', y 'CreditNotPayed'. 
  
* La columna 'Salary' parece ser de tipo objeto, lo cual podría indicar que los datos no están uniformemente formateados (por ejemplo, números representados como texto, posiblemente con símbolos de moneda o separadores de miles). Será necesario convertir esta columna a un formato numérico para su análisis.

In [2]:
# verifico contenido de atributo
for x in df["Salary"].unique(): print(x)

$60K - $80K
Less than $40K
$80K - $120K
$40K - $60K
$120K +
Unknown
nan


**Verifico y elimino valores nulos**

In [3]:

# Verificamos si al menos hay un valor nulo
tiene_nulos = df.isnull().any().any()
print(f"¿El dataset tiene valores nulos? {'Sí' if tiene_nulos else 'No'}")

if tiene_nulos:
    # Eliminación de filas con valores nulos
    df_sin_nulos = df.dropna()
    num_muestras_despues = df_sin_nulos.shape[0]
    print(f"Después de eliminar filas con valores nulos, quedan {num_muestras_despues} muestras.")
else:
    print(f"El dataset original contiene {df.shape[0]} muestras.")


¿El dataset tiene valores nulos? Sí
Después de eliminar filas con valores nulos, quedan 9559 muestras.


**¿Existe algún sesgo por género en el df?**

In [4]:

distribucion_genero = df['Gender'].value_counts(normalize=True) * 100

print("Distribución de género en el dataset (%):")
print(distribucion_genero)

Distribución de género en el dataset (%):
Gender
F    52.905351
M    47.094649
Name: proportion, dtype: float64


Con una distribución de aproximadamente 53% de mujeres y 47% de hombres y en el contexto del problema planteado por el banco, la distribución de género no sugiere inmediatamente un sesgo significativo que afectaría la capacidad de analizar o predecir la fuga de clientes (churn). Sin embargo, la verdadera pregunta sobre la representatividad y el sesgo en este contexto es cómo estas características de género (y otras demográficas) se correlacionan con la probabilidad de cerrar cuentas, y si el modelo predictivo final no perpetúa ni se ve afectado por sesgos inadvertidos.

<br>

**Técnicas para Manejar Categorías "Unknown"**

* **Eliminación**: Si solo unas pocas muestras tienen "Unknown" como valor, se podría optar por eliminarlas. Sin embargo, esto podría no ser ideal si se pierde mucha información.
* **Imputación por la Categoría más Frecuente**: Es útil cuando se pueden asumir que los datos faltantes son aleatorios.
* **Categoría Propia**: En algunos casos, "Unknown" podría considerarse como una categoría propia si representa un grupo significativo y distinto de observaciones.
* **Modelado para Imputación**: Se podría utilizar modelos predictivos para imputar los valores "Unknown" basándote en otras variables del dataset.

Uso de SimpleImputer en sklearn: Específicamente diseñado para la imputación, SimpleImputer puede reemplazar fácilmente los valores faltantes por la media, mediana, la moda (para categóricas) u otro valor constante.


<br>

**Sustitución de "Unknown" por la Categoría más Frecuente**



In [5]:
# Para cada columna categórica, reemplaza 'Unknown' por la moda
#
# for column in df.select_dtypes(include=['object']).columns:
#    most_frequent = df[column].mode()[0]
#    df[column] = df[column].replace('Unknown', most_frequent)

In [6]:
from sklearn.impute import SimpleImputer
import numpy as np

# Reemplaza 'Unknown' por np.nan para que SimpleImputer pueda procesarlo
df.replace('Unknown', np.nan, inplace=True)

# Crea un imputer para reemplazar los valores faltantes por la categoría más frecuente
imputer = SimpleImputer(strategy='most_frequent')

for column in df.select_dtypes(include=['object']).columns:
    # La salida de fit_transform necesita ser adaptada para ser unidimensional
    df[column] = imputer.fit_transform(df[[column]]).ravel()

La elección entre usar *pandas* directamente o *SimpleImputer* de [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) puede depender de tu flujo de trabajo específico y si planeas integrar este paso en un pipeline de preprocesamiento más complejo, para lo cual SimpleImputer podría ser más conveniente.

<br>

**Modifico nombre de las categorías en columna Gender**

In [7]:
df['Gender'] = np.where(df['Gender'] == 'M', 'Masc', np.where(df['Gender'] == 'F', 'Fem', df['Gender']))

# Muestra las primeras filas para verificar los cambios
print(df['Gender'].unique())

['Masc' 'Fem']


**Distribución de las variables**

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# i) Distribución de clientes que se han ido y se han quedado por sexo
gender_churn_hist = px.histogram(df, x='Gender', color='ClosedAccount', 
                                 barmode='group', title="Distribución de Clientes por Sexo")
gender_churn_hist.update_layout(
    height=400,
    width=530,
    showlegend=True
)
gender_churn_hist.show()

# Creación de la primera figura de subplots
fig1 = make_subplots(rows=1, cols=2, subplot_titles=("Distribución de las Edades", 
                                                     "Distribución de las Edades por Sexo"))

# ii) Distribución de las edades de los clientes
age_hist = px.histogram(df, x='Age', nbins=50)
for trace in age_hist.data:
    fig1.add_trace(trace, row=1, col=1)

# iii) Distribución de las edades de los clientes por sexo
age_gender_hist = px.histogram(df, x='Age', color='Gender', barmode='overlay', nbins=50)
for trace in age_gender_hist.data:
    fig1.add_trace(trace, row=1, col=2)

fig1.update_layout(height=400, width = 900, showlegend=True, 
                   title_text="Análisis de Clientes del Banco - Edades y Sexo")
fig1.show()

# Creación de la tercera figura de subplots
fig2 = make_subplots(rows=1, cols=2, subplot_titles=("Distribución de Clientes por Educación", 
                                                     "Distribución de Clientes por Producto"))

# iv) Distribución de clientes que se han ido y que se han quedado por educación
education_hist = px.histogram(df, x='Education', color='ClosedAccount', barmode='group')
for trace in education_hist.data:
    fig2.add_trace(trace, row=1, col=1)

# v) Distribución de clientes que se han ido y se han quedado por el producto asociado
product_hist = px.histogram(df, x='Product', color='ClosedAccount', barmode='group')
for trace in product_hist.data:
    fig2.add_trace(trace, row=1, col=2)

fig2.update_layout(height=400, width = 900, showlegend=True, 
                   title_text="Análisis de Clientes del Banco - Educación y Producto")
fig2.show()

# vi) Distribución de clientes por rango de ingresos y estado de la cuenta
salary_hist = px.histogram(df, x='Salary', color='ClosedAccount', barmode='group',
                            title="Distribución de Clientes por Rango de Ingresos")
salary_hist.update_layout(
    height=350,
    width=950,
    showlegend=True
)
salary_hist.show()


**Distribución de las Edades**:  
La mayoría de los clientes se concentran en un rango de edad de aproximadamente 40 a 60 años. Esto podría indicar que el banco atrae a clientes que están en la mitad de su carrera profesional o se están acercando a la jubilación, lo que puede estar relacionado con la necesidad de servicios financieros relacionados con la planificación del retiro.

**Distribución de las Edades por Sexo**:  
La distribución por edades parece ser similar entre los clientes masculinos y femeninos del banco.

**Distribución de Clientes por Sexo**:  
Hay una cantidad comparable de clientes de cada sexo, pero parece que hay más clientes femeninos que cierran cuentas en comparación con los masculinos. Esto podría ser un área para investigar más a fondo y entender las razones subyacentes de este comportamiento.

**Distribución de Clientes por Rango de Ingresos**:  
La mayoría de los clientes que han cerrado cuentas parecen estar en el segmento de ingresos más bajos. Esto podría indicar problemas de retención de clientes en segmentos de menores ingresos o podría reflejar el comportamiento natural de ese segmento demográfico.

**Distribución de Clientes por Educación**:  
Los clientes con educación "Graduate" parecen ser la mayoría, y entre estos, los que han cerrado cuentas son una porción menor. Esto podría sugerir que el banco retiene mejor a los clientes con niveles de educación más altos.

**Distribución de Clientes por Producto**:  
El producto 1 parece ser el más popularidad, pero también es el producto con el mayor número de clientes que han cerrado sus cuentas.

---
<br>
<br>

**Analizamos por rango de edades de los clientes**


Vamos a convertir la variable `age` de integer a categórica en una columna nueva con rangos de 5 en 5

In [12]:

# Convierte 'Age' a una columna categórica con rangos de edad como strings
df['AgeRange'] = pd.cut(
    df['Age'],
    bins=range(int(df['Age'].min()), int(df['Age'].max()) + 5, 5),
    right=False
).astype(str)  # Convirtiendo a string

df = df.sort_values(by='AgeRange')

# Muestra los primeros registros para verificar la nueva columna
print(df[['Age', 'AgeRange']].head())


       Age  AgeRange
1445  29.0  [26, 31)
3435  28.0  [26, 31)
1573  30.0  [26, 31)
1146  30.0  [26, 31)
1538  27.0  [26, 31)


In [11]:

# Creación de la primera figura de subplots
fig1 = make_subplots(rows=1, cols=2, subplot_titles=("Distribución de las Edades", "Distribución de las Edades por Sexo"))

# i) Distribución de las edades de los clientes con rangos
age_range_hist = px.histogram(df, x='AgeRange')
for trace in age_range_hist.data:
    fig1.add_trace(trace, row=1, col=1)

# ii) Distribución de las edades de los clientes por sexo con rangos
age_range_gender_hist = px.histogram(df, x='AgeRange', color='Gender', barmode='overlay')
for trace in age_range_gender_hist.data:
    fig1.add_trace(trace, row=1, col=2)

fig1.update_layout(height=400, width=900, showlegend=True,
                   title_text="Análisis de Clientes del Banco - Edades y Sexo")
fig1.show()


Parece que hay un número significativamente más alto de clientes con un rango de ingresos "Menos de $40K" que han cerrado sus cuentas en comparación con otros rangos de ingresos. Este hallazgo puede sugerir que el banco podría estar perdiendo clientes en el segmento de ingresos más bajos. Esto podría ser debido a la insatisfacción con los servicios, mejores ofertas de la competencia, o necesidades financieras cambiantes que no están siendo satisfechas por los productos del banco.

**Posibles dos soluciones**

Podría ser útil para el banco investigar por qué los clientes de ingresos más bajos están cerrando sus cuentas y si hay medidas que podrían implementarse para mejorar la retención de clientes en este segmento. Por ejemplo, revisar la estrategia de precios, las tarifas, el tipo de productos ofrecidos, y si estos están adaptados a las necesidades de los clientes de este segmento. Además, el banco podría considerar si está proporcionando un soporte adecuado para la educación financiera y el asesoramiento a los clientes con menos ingresos, lo que podría influir en su decisión de permanecer con el banco.